In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV   #Perforing grid search
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

In [2]:
# 데이터불러오기 ( 2007 - 2017)
d=[]
for year in range(7,18):
    if year//10==0:
        data=pd.read_sas('c:/users/nb-0242/desktop/PHR/질병관리본부/HN'+str(0)+str(year)+'_ALL.sas7bdat', encoding='latin-1')
        d.append(data)
    else :
        data=pd.read_sas('c:/users/nb-0242/desktop/PHR/질병관리본부/HN'+str(year)+'_ALL.sas7bdat', encoding='latin-1')
        d.append(data)
len(d)

11

In [25]:
X_train.shape

(3764, 268)

In [30]:
list(pd.Series(xgb1.get_booster().get_fscore()).sort_values(ascending=False)[:100].keys())

['age',
 'HE_HB',
 'HE_Usg',
 'HE_WBC',
 'BO2_2',
 'N_FE',
 'N_DIET',
 'HE_Upro',
 'HE_RBC',
 'HE_ast',
 'D_1_1',
 'LS_KIND1',
 'HE_PLS',
 'HE_alt',
 'HE_insulin',
 'HE_TG',
 'house_mm',
 'N_WATER',
 'HE_DMdr',
 'HE_HCT',
 'HE_wc',
 'BE5_2',
 'HE_HDL_st2',
 'HE_chol',
 'N_CA',
 'HE_Frtn',
 'HE_crea',
 'HE_Ubld',
 'HE_dbp_tr',
 'ainc_1',
 'BD1_11',
 'N_FIBER',
 'LQ4_07',
 'N_NIAC',
 'HE_wt',
 'N_VA',
 'N_CHO',
 'HE_ht',
 'HE_Uph',
 'N_WAT_C',
 'HE_sbp1',
 'N_RETIN',
 'ainc',
 'HE_BUN',
 'HE_BMI',
 'house_prc',
 'D_2_wk',
 'HE_HPdg',
 'LS_FUN',
 'HE_sbp_tr',
 'N_NA',
 'kstrata',
 'N_B2',
 'N_CAROT',
 'HE_dbp1',
 'BM1_5',
 'LK_EDU',
 'BE3_32',
 'LQ1_mn',
 'LQ4_04',
 'suprt_2',
 'N_ASH',
 'N_B1',
 'relat',
 'LK_GD_P7',
 'BM12_1',
 'N_K',
 'HE_HBsAg',
 'HE_sbp',
 'OR1_2',
 'N_FAT',
 'N_INTK',
 'N_PROT',
 'marri_2',
 'house_yy',
 'N_EN',
 'HE_sbp2',
 'LQ4_03',
 'BO1_1',
 'BP8',
 'BD2',
 'OR3',
 'allownc',
 'LS_FQ1',
 'BE3_11',
 'L_OUT_FQ',
 'BD2_1',
 'N_PHOS',
 'BM1_4',
 'O_DMFTP',
 'BP1',
 

In [39]:
d7 = garbage_killer(d[0])

origin shape : (4594, 1101)
after shape : (1306, 937)


c:\users\nb-0242\anaconda3\envs\ximz\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [42]:
_ = feat_generator(d7,'7xgb')

In [41]:
#method()
def feat_generator(d7_,name):
    feat_list =[]
    DE1_var = list(filter(re.compile('^DE1_.*').match, d7_.columns))
    dn_var = ['HE_glu2', 'HE_fst', 'DE1_dg', 'DE1_31', 'DE1_32', 'HE_DM','HE_DMdg','HE_glu','HE_Uglu','N_DIET_WHY']
    dn_var.extend(DE1_var)
    d_var = list(set(dn_var))

    for i in range(100):
        X_train, X_test, y_train, y_test = train_test_split(d7_.iloc[:,1:], d7_['DE1_dg'],test_size=0.2)
        X_train = X_train.iloc[:, ~X_train.columns.isin(d_var)]
        X_test = X_test.iloc[:, ~X_test.columns.isin(d_var)]
        xgb_ = XGBClassifier()
        xgb_.fit(X_train, y_train)
#         xgb_yhat = xgb_.predict(X_test)
        feat_importances = pd.Series(xgb_.get_booster().get_fscore()).sort_values(ascending=True)
        feat_list.append(list(feat_importances[:101].keys()))

    feat_list_ = sum(feat_list, [])
    uni, cnt = np.unique(feat_list_, return_counts=True)
    feat100 = pd.DataFrame(data=cnt,index=uni)
    feat100.sort_values(by=0,ascending=False,inplace=True)
    feat100.to_csv('c:/users/nb-0242/jupyter/PHR/pp_xgb/feat'+name+'_100.csv', index=True)
    return feat100

In [37]:
def garbage_killer(d):
    # 여성관련 변수 제거
    r = re.compile('^HW_.*')
    w_var =list(filter(r.match, d.columns))
    d_ = d.iloc[:, ~d.columns.isin(w_var)]
    
    # 가중치 변수 제거
    r2 = re.compile('^wt_.*')
    weight_var = list(filter(r2.match, d_.columns))
    d__ = d_.iloc[:, ~d_.columns.isin(weight_var)]
    
    #쓰레기변수 삭제
    g_var = [x for x in d__.columns if str(d__[x].dtypes) not in ['float64', 'int64']]
    d__ = d__.iloc[:, ~d__.columns.isin(g_var)]
    
    # DE1_dg 보다 결측치 많은 변수 제거 ㅎ , default 2000
    dc1_null = d__['DE1_dg'].isnull().sum()
    if dc1_null > 2000:
        
        d_ = d__.iloc[:, d__.isnull().sum().values <dc1_null+1]
    else :
        d_ = d__.iloc[:, d__.isnull().sum().values < 2000 ]
    d_.dropna(inplace=True)
    print('origin shape : {}'.format(d.shape))
    print('after shape : {}'.format(d_.shape))
    return d_

In [ ]:
for idx in range(1,11):
    d_year= garbage_killer(d[idx])
    _ = feat_generator(d_year, name = str(idx+7)+'xgb')

c:\users\nb-0242\anaconda3\envs\ximz\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


origin shape : (9744, 1159)
after shape : (5834, 224)
origin shape : (10533, 1195)
after shape : (7438, 890)
